In [ ]:
# !pip install optuna

In [ ]:
import optuna
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
import sys
# Append the directory of the script to sys.path
sys.path.append("../src/ITM_Classifier-baseline")

# Now you can import the classes
from ITM_Classifier_baseline import ITM_DataLoader, ITM_Classifier

In [ ]:
def create_model(trial):
    # Define the hyperparameter search space using the trial object
    num_layers = trial.suggest_int('num_layers', 1, 3)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    
    # Assuming you have modified ITM_Classifier to accept these parameters
    itm_classifier = ITM_Classifier(num_layers=num_layers, dropout_rate=dropout_rate)
    itm_classifier.learning_rate = learning_rate  # Setting learning rate
    return itm_classifier

def objective(trial):
    # Create the model for this trial
    itm_classifier = create_model(trial)
    
    # Train the model (You may want to limit the number of epochs or use early stopping)
    itm_classifier.train_classifier_model()
    
    # Evaluate the model performance
    loss, accuracy = itm_classifier.classifier_model.evaluate(itm_classifier.val_ds)
    
    # Optuna aims to minimize the objective, so if accuracy is your goal, return 1-accuracy
    return 1 - accuracy

In [ ]:
def main():
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=20)  # Specify the number of trials

    print('Best trial:')
    trial = study.best_trial

    print(f'  Value: {1 - trial.value}')
    print('  Params: ')
    for key, value in trial.params.items():
        print(f'    {key}: {value}')

In [ ]:
if __name__ == "__main__":
    main()